In [1]:
using Pkg
Pkg.activate("EnvFerrite")
include("EITwithTV.jl")

  Activating project at `~/Code/Julia/FerriteStuff/Notebooks/Github/EnvFerrite`


update_Jr (generic function with 1 method)

# Electrical Impedance Tomography with TV Regularization

## Introduction

In this example we give a basic implementation of of the real valued Calderon problem relevant to Electrical Impedance Tomography. We will generate data and afterwards solve the inverse problem with a numerical solver and implement TV regularization.  

### Forward EIT:
Given a conductivity $\gamma: \Omega\subset\mathbb{R}^2 \rightarrow \mathbb{R}_{+}$ our solution $u\in H^1(\Omega,\mathbb{R}^2)$ (2D case) has to confirm to the equation:


$$ \nabla \cdot(\gamma\nabla u) = 0 \quad \forall x \in \Omega $$

For that equation we will choose a set of electrical current patterns 

$$ g_1, ..., g_n \in H^{-\frac{1}{2}}(\partial \Omega, \mathbb{R}) $$

such that:  

$$ \int_{\partial\Omega}g_i \, d\partial\Omega = 0 $$

to inject into the material via Neumann boundary conditions:

$$ \gamma\frac{\partial u_i}{\partial n} =g_i \quad \forall x \in \partial\Omega $$

In order to get the corresponding voltages 
$$ f_1, ..., f_n \in H^{\frac{1}{2}}(\partial \Omega, \mathbb{R}) $$
we will measure the corresponding voltage as:
$$ f_i := u_i|_{\partial\Omega} $$

with those boundary pairs $ (f_1,g_1), ... , (f_n,g_n) $ we now have an approximation of the Dirichlet to Neumann map:
$$ \Lambda_\gamma: H^{\frac{1}{2}}(\partial \Omega)\rightarrow H^{-\frac{1}{2}}(\partial \Omega) $$
This is called forward EIT since we just approximated the map:
$$ \gamma \rightarrow  \Lambda_\gamma $$

However real Electrical Impedance Tomography requires us to solve an **inverse Problem** where we have to reconstruct:
$$ \Lambda_\gamma \rightarrow  \gamma $$
for our approximation of $\Lambda_\gamma$ given by voltage-current boundary pairs $(f_i,g_i)$.

### Weak formulation
Given the strong formulation:
$$ \nabla \cdot(\gamma\nabla u) = 0 \quad \text{with Neumann BC:}\quad \gamma\frac{\partial u}{\partial n} = g $$

The weak formulation is:
$$ \int\limits_\Omega \gamma \nabla(u)\cdot\nabla(v) \, d \Omega = \int\limits_{\partial\Omega} g \,v \,d\partial\Omega  \quad \forall v\in H^1(\Omega)$$

### Inverse EIT
We will use $\gamma$ to refer to the true underlying conductivity and $\sigma$ for our current conductivity guess.
We will choose the simplest minimization functional for optimization:
$$ J_i(u_i,\sigma) = \| f_i- u_i\|^2_{\mathcal{L}^2(\partial\Omega)} = \int_{\partial\Omega}(f_i-u_i)^2 \,d\partial\Omega $$
In theory however we can plug in another metric or pseudo-metric like Wasserstein-distance or Spetral distance as mesure of distance between $f_i$ und $u_i$.
Such that our problem becomes:
$$ \underset{\sigma}{\min} \sum\limits_{i=1}^n J_i(u_i,\sigma) $$
such that:
$$ \nabla u_i\cdot(\sigma\nabla u_i) = 0\quad  \text{and Neumann BC}\quad \sigma\frac{\partial u_i}{\partial n} = g_i \quad \forall i\in\{1, ...,n\}$$
Given the problem our lagrangian becomes:
$$ \mathcal{L}(\sigma, u, \lambda) = \sum\limits_{i=1}^n\left( J_i(u_i,\sigma) + \langle \lambda_i, \nabla\cdot(\sigma\nabla u_i) \rangle_{\mathcal{L}^2(\Omega)}   \right) $$

from this we will use [Adjoint state methods](https://en.wikipedia.org/wiki/Adjoint_state_method#General_case) to calculate the gradient.
Without stating any of the steps of the derivation we end up with:
- State Equation (Variation with $\delta_\lambda$)
$$ \nabla \cdot(\sigma\nabla u_i) = 0\quad  \text{with Neumann BC}\quad \sigma\frac{\partial u_i}{\partial n} = g_i$$

- Adjoint Equation (Variation with $\delta_u$)
$$ \nabla \cdot(\sigma\nabla\lambda_i) = 0\quad  \text{with Neumann BC}\quad \sigma\frac{\partial \lambda_i}{\partial n} = 2(u_i-f_i)$$
- Functional Derivative (Variation with $\delta_\sigma$)
$$ \delta_i\sigma = -\nabla u_i \cdot \nabla \lambda_i $$

#### Total Variation (TV) regularization
for real world examples and numerical stability we have to assume that our system contains some noise, like $f = f_{true} + \epsilon$. Since the inverse EIT problem is highly ill conditioned we have to consider regularization.


Because $|\nabla\sigma|^2$ is non-differentiable when $\nabla\sigma=0$, we use
$$
\mathcal{R}_{TV}(\sigma) = \int_\Omega \sqrt{|\nabla\sigma|^2+\eta}\, d\Omega,
$$
and the gradient:
$$ \delta_\sigma\mathcal{R}= -\nabla\cdot \left(   \frac{\nabla\sigma}{\sqrt{|\nabla \sigma|^2+ \eta}} \right) $$
with a small $\eta$ to revent division by zero.
This is a $L^2$ projection that requires us to sove the weak form.
##### Weak formulation of TV Regularizer:
$$ \int_\Omega wv \,d\Omega = \int_\Omega \frac{\nabla(\sigma)}{\sqrt{|\nabla\sigma|^2+\eta}}\cdot\nabla(v)\, d\Omega \quad \forall v\in FESpace$$


### Full reconstruction Algorithm (Conceptual)
+ Preallocate Massmatrix M and L^2 projector (Cholesky factorization)
+ Start with conductivity guess $\sigma_0$ (In our case: $\sigma_0(x) = 1.0$)
+ Preallocate & initialize Conjugate Gradient(CG) solver for $u_1, ...,u_n.\lambda_1,.., \lambda_n,w$.
+ Repeat till tolerance is reached or other stopping condition:
    + From $\sigma_t$ assemble stiffness matrix $K_{\sigma_t}$
    + for all $i = 1, ...,n$ (in parallel)
        + Calculate $u_i$ (State equation) as well as the $\mathcal{L}^2(\partial\Omega)$ error: $\delta u_i$
        + Calculate $\lambda_i$ (Adjoint equation)
        + Calculate $\delta\sigma_i$ (Functional derivative)
    + Calculate TV Regularization gradient and error.
    + Update $\sigma_{t+1} = \sigma_t +\beta\, \delta_{TV}\sigma + \sum_{i=1}^n \alpha_i\,\delta_i\sigma $ (with Gauss-Newton with Levenberg-Marquardt).

## Implementation
### Preliminaries
Obvious Imports:

In [2]:
using Ferrite
using SparseArrays
using LinearAlgebra
using Revise
using Interpolations
using Plots
using Statistics
using IterativeSolvers
using LinearMaps

For simplicity we will use a quadratic grid with quadrilateral elements. We are using Quadrilaterals for now:

In [3]:
grid = generate_grid(Quadrilateral, (32, 32));
dim = Ferrite.getspatialdim(grid)
order = 1


ip = Lagrange{RefQuadrilateral, order}()
qr = QuadratureRule{RefQuadrilateral}(2)
qr_face = FacetQuadratureRule{RefQuadrilateral}(2)
cellvalues = CellValues(qr, ip)
facetvalues = FacetValues(qr_face, ip)

dh = DofHandler(grid)
add!(dh, :u, ip)
close!(dh)



∂Ω = union(getfacetset.((grid,), ["left", "top", "right", "bottom"])...)
length(∂Ω)

128

For later use we will assemble and cholesky decompose the mass matrix once.

In [4]:
# This is supposed to be: ∫(u*v)dΩ and it's Cholesky decomposition
M, MC = assemble_M(cellvalues,dh)

(sparse([1, 2, 3, 4, 1, 2, 3, 4, 5, 6  …  1054, 1055, 1056, 1087, 1088, 1089, 1055, 1056, 1088, 1089], [1, 1, 1, 1, 2, 2, 2, 2, 2, 2  …  1088, 1088, 1088, 1088, 1088, 1088, 1089, 1089, 1089, 1089], [0.0004340277777777777, 0.00021701388888888885, 0.00010850694444444441, 0.00021701388888888882, 0.00021701388888888885, 0.0008680555555555555, 0.0004340277777777777, 0.00010850694444444441, 0.00021701388888888893, 0.00010850694444444444  …  0.00010850694444444444, 0.0004340277777777777, 0.00010850694444444441, 0.00021701388888888882, 0.0008680555555555553, 0.00021701388888888882, 0.00010850694444444441, 0.00021701388888888882, 0.00021701388888888882, 0.00043402777777777765], 1089, 1089), SparseArrays.CHOLMOD.Factor{Float64, Int64}
type:    LLt
method:  simplicial
maxnnz:  20895
nnz:     20895
success: true
)

Furthermore we want to know which entries of the force vector correspond to the boundary:
We will get:
- the count of nonzero entries in the force vector
- the position of non zero entries
- a function "up" to cast a vector of length of boundary dofs into the length of the force vector
- a function "down" to cast a vector into the length of the dofs of the force vector that lay on the boundary.

Note: depending on the grid and the method used this can be more complicated up and down function.
I want to from the start take a very modular approach where one can plugin different methods, even Spectral Methods instead of FEM and the Code might be reusable. That is why in some cases up and down wouldn't be a simple function in vector indices but a full blown operator.

In [5]:
nzc,nzpos, down, up = produce_nonzero_positions(facetvalues, dh,∂Ω)
@assert nzc == length(∂Ω)  # This is not true in Gridap.jl

In [6]:
## Sanity check:
# I have never questioned the assumption that up∘down == id and down∘up == id. Maybe I should check this.
test_vec = [i for i in 1:nzc]
@assert down(up(test_vec)) == test_vec
@assert up(down(up(test_vec))) == up(test_vec)

### Data generation

Now we will make up some conductivity. As well as some current patterns:

In [7]:
conductivity  = (x) -> 1.1 + sin(x[1]) * cos(x[2])

#24 (generic function with 1 method)

For later we want to project that function unto Q1 FE space for that we want to assemble the coefficients in the FESpace:

In [8]:

cond_vec = assemble_function_vector(cellvalues, dh, conductivity, M)

1089-element Vector{Float64}:
 0.6450535642159253
 0.6641859150714466
 0.6226468097664015
 0.601690881462597
 0.685021948586164
 0.645468805231958
 0.707477970024696
 0.6700651958589253
 0.7314669135515499
 0.6963406172344299
 ⋮
 1.3592050021692093
 1.3883340781632245
 1.416337215072183
 1.4431050644646248
 1.4685330864484494
 1.492522029975305
 1.5149780514138347
 1.5358140849285546
 1.5549464357840737

In [9]:
# This function assembles the stiffness matrix from a given vector.
# This is: ∫(γ * ∇(u)⋅∇(v))dΩ 
K_from_vec = assemble_K(cellvalues, dh,cond_vec)

1089×1089 SparseMatrixCSC{Float64, Int64} with 9409 stored entries:
⎡⠻⣦⡸⣆⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎤
⎢⠲⢮⣿⣿⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠈⠻⣿⣿⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠈⠻⣿⣿⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠈⠻⣿⣿⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣟⣽⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣿⣿⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣿⣿⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣟⣽⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣿⣿⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣿⣿⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣟⣽⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣿⣿⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣿⣿⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣟⣽⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣿⣿⣦⡀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣿⣿⣦⡀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣿⣿⣦⡀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣿⣿⣦⡀⎥
⎣⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠻⣿⣿⎦

In [10]:
# This is matrix assembly on a function. How do I do it if the conductivity is given as a coefficient vector for Q1 FE Space?
# This is: ∫(γ * ∇(u)⋅∇(v))d
K_true = assemble_K(cellvalues, dh, conductivity)
K_true_LU = factorize(K_true)

SparseArrays.CHOLMOD.Factor{Float64, Int64}
type:    LLt
method:  simplicial
maxnnz:  20895
nnz:     20895
success: true


In [11]:
# Sanity check: (positive semidefinite self adjoint stiffness matrix)
@assert K_true == K_true'  # Not true in Gridap.jl
if ndofs(dh) < 500
    K_dense = Matrix(K_true)
    eig_min = minimum(eigvals(K_dense))
    @assert eig_min > -1e-14
    print("Smallest eigenvalue: ", eig_min)
end

Now we generate current patterns: We assume one current source and one current sink. Important is that it sums of to zero.
We generate the right hand side force vectors $g_1, ... g_n$ as an Matrix G and calculate $f_1, ..., f_n$

In [12]:
num_modes = Int64((nzc^2-nzc)//2)
G_small = zeros(nzc,num_modes)
G = zeros(ndofs(dh),num_modes)
k = 1
for i in 1:(nzc-1)
    for j in i+1:nzc
        G_small[i,k] = 1.0
        G_small[j,k] = -1.0
        G[:,k] = up(G_small[:,k])
        k += 1
    end
end
F_big = K_true_LU \ G
F = zeros(nzc,num_modes)
k = 1
for i in 1:(nzc-1)
    for j in i+1:nzc
        F[:,k] = down(F_big[:,k])
        k += 1
    end
end
col_means = mean(F, dims=1)
F .-= col_means

128×8128 Matrix{Float64}:
  2.28031     2.25774       2.8858     …   0.00896351   0.00471518
 -0.822377    0.128708      0.503204       0.00896068   0.00471448
  0.15127    -0.907189      0.708219       0.00896638   0.0047159
 -0.102286    0.0801673    -1.03945        0.00895235   0.00471241
 -0.0645602   0.030441     -0.242412       0.00893867   0.00470901
 -0.0399858   0.0134256    -0.147098   …   0.00891984   0.00470433
 -0.0299563   0.00458881   -0.0979698      0.00889602   0.0046984
 -0.0244935  -0.000326226  -0.074219       0.00886739   0.00469129
 -0.0212772  -0.00332372   -0.0605148      0.00883415   0.00468302
 -0.0192336  -0.00526549   -0.0519835      0.0087965    0.00467366
  ⋮                                    ⋱               
 -0.0134128  -0.0109905    -0.0283843      0.0240971    0.00839202
 -0.013424   -0.0109791    -0.0284286  …   0.0284299    0.00941065
 -0.0134336  -0.0109693    -0.0284667      0.0352115    0.0109726
 -0.0134417  -0.0109611    -0.0284985      0.04667

To be realistic we will add some noise:
We have to ensure that our noise has mean zero.

In [13]:
function mean_zero_noise(n::Int64, σ::Float64)
    out = σ * randn(n)
    mean = Statistics.mean(out)
    out .- mean
end

mean_zero_noise (generic function with 1 method)

To simplify things we can also do SVD:

In [14]:
# reduce the number of modes according to used SVD modes

F, Σ, G_small, Λ, num_modes = do_svd(F,G_small)
# Apply singular Values:
G = zeros(ndofs(dh),num_modes)
F_big = copy(G)
for i in 1:num_modes
    G[:,i] = up(G_small[:,i])
    F_big[:,i] = up(F[:,i])
end

now that we have done SVD on the original choice of modes we have 
- Truncation of SVD modes, thus regularization
- Averaging of noise over multiple measurements
- Elimination of any unncessary number of nodes we choose before.

Here we define a struct where we save and preallocate all the necessary information for the solver step.

In [15]:
# Implement a sanity check if the two matrices assembled from the function and the vector are roughly the same (use relatively coarse ≈ )
Matrix_norm = norm(K_from_vec - K_true)
println("Norm of Matrix difference: ",Matrix_norm)
@assert Matrix_norm < 20.0

g_test = G[:,1]
f_test_true = K_true \ g_test
f_test_vec = K_from_vec \g_test
vector_norm = norm(f_test_true - f_test_vec)
println("norm of difference of first SVD mode: " ,vector_norm)
@assert vector_norm < 13.0

Norm of Matrix difference: 8.536307354525758e-6
norm of difference of first SVD mode: 11.472224696560371


In [16]:
mode_dict = Dict{Int64,EITMode}()
for i in 1:num_modes
    mode_dict[i] = EITMode(G[:,i],F[:,i])
end

### Solving EIT

We will now assume a starting conductivity guess $\sigma_0(x) = 1.0 $

In [17]:
σ₀ = (x) -> 1.0

#26 (generic function with 1 method)

We would prefer to save $\sigma$ as a vector for use in FEM and also have a method to export each

In [18]:
# Project function here: 
σ = assemble_function_vector(cellvalues,dh, σ₀, MC)


1089-element Vector{Float64}:
 0.9999999999999998
 1.0
 0.9999999999999996
 1.0000000000000002
 0.9999999999999989
 1.0000000000000007
 1.0000000000000002
 0.9999999999999992
 0.9999999999999996
 1.0000000000000009
 ⋮
 1.0
 1.0000000000000004
 1.0
 1.0000000000000009
 0.9999999999999997
 1.0000000000000007
 0.9999999999999998
 1.0000000000000002
 0.9999999999999997

A prerequisite is that we can calculate the bilinear map: $\nabla(u)\cdot\nabla(\lambda)$


In [19]:
# Assemble right-hand side for the projection of ∇(u) ⋅ ∇(λ) onto the FE space.
# This computes rhs_i = ∫ (∇u ⋅ ∇λ) ϕ_i dΩ for each test function ϕ_i.
# Assuming u and λ are scalar fields in the same FE space.
# cellvalues should be CellScalarValues(qr, ip) where qr is QuadratureRule, ip is Interpolation.
#function calculate_bilinear_map(a::AbstractVector, b::AbstractVector, cellvalues::CellValues, dh::DofHandler, M_cholesky)
# Write some code which demonstrates this function:


Here we define which metric we want to use:
In our case we will stick with the squared $L^2$ metric, but in theory one would receive more stable EIT reconstruction if instead one calculates Wasserstein distance, Spectral distances or similar.

In [20]:
# Here we define the metric we will use:
d = (x,y) -> norm(x-y)^2
∂d = (x,y) -> 2*(x-y) 
# This allows us to plugin other metrics and differentiate with Enzyme:
# define metric here:

# define other metric here:
#d = (x, y) -> ...
# We can now use Enzyme or Zygote Autodiff library:
using Enzyme
makegradₓ(d) = (x, y) -> Enzyme.gradient(Reverse, Const(d), x, y)[1]
∂ₓd = makegradₓ(d)
# Sanity test, whether this is working:
a = randn(10)
b = randn(10)
@assert norm(∂d(a,b) - ∂ₓd(a,b)) ≈ 0

┌ Warning: Using fallback BLAS replacements for (["dasum_64_"]), performance may be degraded
└ @ Enzyme.Compiler ~/.julia/packages/Enzyme/sQTaL/src/compiler.jl:4430


With the given matrix and projector our we need to solve for every mode $(f_i,g_i)$ the adjoint-state-solution to get the gradient:

$$ \nabla_\sigma d(K_\sigma\; g_i,f_i) $$
with $d(\cdot,\cdot)$ being some pseudo metric through which we measure the error and $K_\sigma$ the Forward EIT operator dependent on $\sigma$
Our implementation becomes:
```julia
function state_adjoint_step!(mode::EITMode, K_factorized, M, d,∂d ,down,up,dh::DofHandler, cellvalues::CellValues)
    # We solve the state equation ∇⋅(σ∇uᵢ) = 0 : σ∂u/∂𝐧 = g
    mode.u = K \ mode.g
    # Projection from down:Ω → ∂Ω
    b = down(mode.u) 
    # Normalize: ∫(uᵢ)d∂Ω = 0
    mean = Statistics.mean(b) 
    b .-= mean
    mode.u .-= mean 
    # We solve the adjoint equation ∇⋅(σ∇λᵢ) = 0 : σ∂u/∂𝐧 = ∂ₓd(u,f)
    mode.λ = K \ up(∂d(b,mode.f)) 
    # Note: we have projection up: ∂Ω → Ω (fill in zeros)
    # ∂ₓd is gradient of pseudo metric d(x,y)
    mode.error = d(b,mode.f) # Error according to pseudo metric d(x,y)
    # Calculate ∇(uᵢ)⋅∇(λᵢ) here: 
    mode.δσ = calculate_bilinear_map(mode.λ, mode.u, cellvalues, dh, M) 
end
```
For efficiency we will use a Conjugate Gradient (CG) solver in production.

In [21]:
# For comparison to adjoint-state-method implement gradient based on Autodiff here:


### Regularization
#### Tikhonov Regularization
For Tikhonov regularization we already have assembled:

In [22]:
# This is: ∫(∇(u)⋅∇(v))dΩ the stiffness matrix without specified coefficients. It is used for Tikhonov H¹ regularization. 
K_st , K_TK = assemble_K(cellvalues, dh)

(sparse([1, 2, 3, 4, 1, 2, 3, 4, 5, 6  …  1054, 1055, 1056, 1087, 1088, 1089, 1055, 1056, 1088, 1089], [1, 1, 1, 1, 2, 2, 2, 2, 2, 2  …  1088, 1088, 1088, 1088, 1088, 1088, 1089, 1089, 1089, 1089], [0.6666666666666664, -0.16666666666666657, -0.3333333333333332, -0.16666666666666657, -0.16666666666666657, 1.3333333333333326, -0.33333333333333326, -0.3333333333333332, -0.1666666666666664, -0.33333333333333326  …  -0.3333333333333333, -0.33333333333333326, -0.3333333333333332, -0.1666666666666666, 1.333333333333333, -0.16666666666666657, -0.3333333333333332, -0.16666666666666657, -0.16666666666666657, 0.6666666666666665], 1089, 1089), SparseArrays.CHOLMOD.Factor{Float64, Int64}
type:    LLt
method:  simplicial
maxnnz:  20895
nnz:     20895
success: true
)

In [23]:
mutable struct FerriteEITProblem
    M # Massmatrix ∫(ϕᵢϕⱼ)dΩ as Sparse matrix
    MC # Massmatrix Cholesky 
    KST # Stiffness matrix ∫(∇ϕᵢ⋅∇ϕⱼ)dΩ of the mesh
    up # Projection to boundary (Can be an operator)
    down # Projection from boundary to force vector
    d # the metric we are using for the problem
    ∂ₓd # the metric d(x,y) differentiated after x
    σ # The current guess of the conductivity
    K # The current guess of the stiffnessmatrix ∫(σ*∇ϕᵢ⋅∇ϕⱼ)dΩ assembled from σ
    Kfac # factorized version of guess of stiffness matrix
    cellvalues::CellValues
    dh::DofHandler
    n::Int64 # ndofs(dh)
end
K_σ = 
data = FerriteEITProblem(M,MC,K_st,up,down, d, ∂d, σ, K, K)

UndefVarError: UndefVarError: `K` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

#### TV regularization
Additinal we need to assemble the TV regularizer. The required Mass matrix we already have asembled and ready to use. 


In [24]:
tv = TV(ndofs(dh))
calc_tv_gradient(σ, tv, cellvalues, dh, M)

([0.00014211640344179044, -0.00018486576361887745, 0.00021187470600005074, -0.00016478845034880395, 0.0003786302204325758, -0.0003455130900355594, -0.00031051581729699086, 0.00039107620269100533, 0.0002953666001322491, -0.00044853071079305917  …  -2.178005776618398e-5, 0.00010370979287720843, -0.0001541848265629342, 9.923186529710191e-5, -0.00030442908176604136, 0.00022535191975102967, -0.000323138138475289, 0.00022995761620118153, -0.00015385376550725737, 0.00021630202062267564], 1.0000000000007197e-8)

For finding suitable stepsizes we will use Gauss-Newton.
Since we want to avoid implementing a dense Hessian Matrix we use SVD to invert the Matrix.

In [25]:
#=
using LinearAlgebra

# Forward operator: F(γ) -> simulated boundary voltages
function forward(γ)
    # placeholder: replace with your FEM/EIT solver
    return γ # dummy
end

# Residual: r(γ) = F(γ) - y
function residual(γ, y)
    return forward(γ) - y
end

# Gradient of data term (Jacobian): ∇_γ 0.5*||F(γ)-y||^2
function grad_data(γ, y)
    # placeholder: replace with actual Jacobian * residual
    return residual(γ, y)
end

# Example proximal operator: soft-thresholding (for L1 norm)
function prox_l1(x, λ)
    return sign.(x) .* max.(abs.(x) .- λ, 0)
end

# Split Bregman iteration
function split_bregman(γ0, y; λ=1.0, μ=1.0, niter=20, prox=prox_l1)
    γ = copy(γ0)
    d = zeros(size(γ0))
    b = zeros(size(γ0))
    
    for k in 1:niter
        # --- Update γ (data term + quadratic penalty) ---
        # Solve: min 0.5||F(γ)-y||^2 + μ/2 ||∇γ - d - b||^2
        # Here we do a simple gradient descent for illustration
        g = grad_data(γ, y) + μ * (γ - d - b) # replace ∇γ with identity for simplicity
        γ -= 0.1 * g # step size 0.1, tune for your problem
        
        # --- Update auxiliary variable d (prox of regularizer) ---
        d = prox(γ + b, λ/μ)
        
        # --- Update Bregman variable ---
        b += γ - d
    end
    
    return γ
end
=#

In [26]:
function full_step!(M,σ::AbstractVector ,modes::Dict{Int64,EITMode}, num_modes::Int64,tv::TV,  d,∂d ,down,up, dh::DofHandler, cellvalues::CellValues, do_TV::Bool =true, β::Float64 = 1e-5)

    

Base.Meta.ParseError: ParseError:
# Error @ /home/daniel/Code/Julia/FerriteStuff/Notebooks/Github/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_Y106sZmlsZQ==.jl:3:5

    
#   └ ── premature end of input

Now that we have all the pieces we can assemble the full optimization step:

In [27]:
# note: If you want to use truncated SVD as regularization one can pass a smaller number than num_modes
function full_step_old!(M,σ::AbstractVector ,modes::Dict{Int64,EITMode}, num_modes::Int64,tv::TV,  d,∂d ,down,up, dh::DofHandler, cellvalues::CellValues, do_TV::Bool =true, β::Float64 = 1e-5)
    # Assemble Matrix: (from vector)
    K = assemble_K(cellvalues,dh,σ)
    if do_TV
        J = zeros(num_modes+1,ndofs(dh))
        r = zeros(num_modes+1)
        # Launch TV regularizer:
        tv_task = Threads.@spawn begin
            calc_TV_step!(σ,tv, dh,cellvalues,M)
        end
    else
        J = zeros(num_modes,ndofs(dh))
        r = zeros(num_modes)
    end
    # solve adjoint state method
    Threads.@threads for i in 1:num_modes
        state_adjoint_step!(mode_dict[i], K, M,  d,∂d , down, up, dh, cellvalues)
    end

    # Fetch gradients & errors
    for i in 1:num_modes
        J[i,:] = mode_dict[i].δσ
        r[i] = mode_dict[i].error
    end
    if do_TV
        # Fetch TV regularization
        fetch(tv_task)
        J[num_modes+1,:] = tv.δ
        r[num_modes+1] = β * tv.error 
    end    
    # calculate steps with Gauss-Newton
    δσ = gauss_newton(J, r, λ=1e-3)
    # update σ
    σ .+= δσ
    σ .= max.(σ ,1e-12) # Ensure positivity
    return δσ,r,J
end

full_step_old! (generic function with 3 methods)

In [28]:
# note: If you want to use truncated SVD as regularization one can pass a smaller number than num_modes
function cheat_step!(M,γ,σ::AbstractVector ,modes::Dict{Int64,EITMode}, num_modes::Int64,tv::TV,  d,∂d ,down,up,dh::DofHandler, cellvalues::CellValues, do_TV::Bool =true, β::Float64 = 1e-5)
    # Assemble Matrix: (from vector)
    K = assemble_K(cellvalues,dh,σ)
    if do_TV
        J = zeros(num_modes+1,ndofs(dh))
        r = zeros(num_modes+1)
        # Launch TV regularizer:
        tv_task = Threads.@spawn begin
            calc_TV_step!(σ,tv, dh,cellvalues,M)
        end
    else
        J = zeros(num_modes,ndofs(dh))
        r = zeros(num_modes)
    end
    # solve adjoint state method
    Threads.@threads for i in 1:num_modes
        state_adjoint_step!(mode_dict[i], K, M,  d,∂d ,down,up,dh, cellvalues)
    end

    # Fetch gradients & errors
    for i in 1:num_modes
        J[i,:] = mode_dict[i].δσ
        r[i] = mode_dict[i].error
    end
    if do_TV
        # Fetch TV regularization
        fetch(tv_task)
        J[num_modes+1,:] = tv.δ
        r[num_modes+1] = β * tv.error 
    end    
    # calculate steps with Gauss-Newton
    δσ = gauss_newton(J, r, λ=1e-3)
    # update σ
    α = - dot(γ-σ, δσ )
    σ .-= α*δσ
    σ .= max.(σ ,1e-12) # Ensure positivity
    return δσ,r,J
end

cheat_step! (generic function with 3 methods)

In [29]:
function full_step_initial!(M,σ::AbstractVector ,modes::Dict{Int64,EITMode}, num_modes::Int64,tv::TV,  d,∂d ,down,up, dh::DofHandler, cellvalues::CellValues)
    # Assemble Matrix: (from vector)
    K = assemble_K(cellvalues,dh,σ)
    K_LU = lu(K)
    J = zeros(num_modes,ndofs(dh))
    r = zeros(num_modes)
    # solve adjoint state method
    Threads.@threads for i in 1:num_modes
        state_adjoint_step!(mode_dict[i], K, M,  d, ∂d ,down , up, dh, cellvalues)
    end

    # Fetch gradients & errors
    for i in 1:num_modes
        J[i,:] = mode_dict[i].δσ
        r[i] = mode_dict[i].error
    end
    # calculate steps with Gauss-Newton
    δσ = gauss_newton(J, r, λ=1e-3)
    # update σ
    σ .+=  δσ
    σ .= max.(σ ,1e-12) # Ensure positivity
    return K,δσ,r,J
end

full_step_initial! (generic function with 1 method)

In [30]:
σ_prev = copy(σ)
error  = norm(cond_vec - σ)
println("With error: ", error)

With error: 15.284988831024412


Let's run this optimization loop a few times:

In [31]:
result, ftime, bytes, gctime, memallocs = @timed begin
 _ ,δσ =  full_step_initial!(MC, σ, mode_dict, num_modes, tv,  d,∂d ,down,up,dh, cellvalues)
end
println("Time: ", ftime, " seconds, Bytes: ", bytes, ", GC time: ", gctime, ", Memory allocations: ", memallocs)

CompositeException: TaskFailedException

    nested task error: UndefVarError: `state_adjoint_step!` not defined in `Main`
    Suggestion: check for spelling errors or missing imports.
    Stacktrace:
     [1] macro expansion
       @ ~/Code/Julia/FerriteStuff/Notebooks/Github/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_Y113sZmlsZQ==.jl:9 [inlined]
     [2] (::var"#138#threadsfor_fun#45"{var"#138#threadsfor_fun#44#46"{SparseArrays.CHOLMOD.Factor{Float64, Int64}, var"#28#29", var"#30#31", var"#14#20"{Vector{Int64}}, var"#15#21"{Vector{Int64}}, DofHandler{2, Grid{2, Quadrilateral, Float64}}, CellValues{Ferrite.FunctionValues{1, Lagrange{RefQuadrilateral, 1}, Matrix{Float64}, Matrix{Vec{2, Float64}}, Matrix{Vec{2, Float64}}, Nothing, Nothing}, Ferrite.GeometryMapping{1, Lagrange{RefQuadrilateral, 1}, Matrix{Float64}, Matrix{Vec{2, Float64}}, Nothing}, QuadratureRule{RefQuadrilateral, Vector{Float64}, Vector{Vec{2, Float64}}}, Vector{Float64}}, SparseMatrixCSC{Float64, Int64}, UnitRa

In [32]:
calc_tv_gradient(σ, tv, cellvalues, dh, M)

([0.0002842328068835809, -0.0003697315272377549, 0.00042374941200010147, -0.0003295769006976079, 0.0007572604408651516, -0.0006910261800711188, -0.0006210316345939818, 0.0007821524053820107, 0.0005907332002644982, -0.0008970614215861183  …  -4.356011553236795e-5, 0.00020741958575441687, -0.0003083696531258684, 0.00019846373059420386, -0.0006088581635320828, 0.0004507038395020594, -0.000646276276950578, 0.00045991523240236306, -0.00030770753101451475, 0.00043260404124535127], 1.0000000000007197e-8)

In [33]:
dot(cond_vec-σ_prev, δσ )

UndefVarError: UndefVarError: `δσ` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [34]:
error  = norm(cond_vec - σ)
println("With error: ", error)

With error: 15.284988831024412


In [35]:

for i in 1:20
    print("Step ", i, " ")
    result, time, bytes, gctime, memallocs = @timed begin
        δσ, _ = full_step!(MC, σ, mode_dict, 10, tv, d,∂d ,down,up,dh, cellvalues, false)
        #δσ, _ = cheat_step!(MC, cond_vec, σ, mode_dict, 10, tv,  d,∂d ,down,up,dh, cellvalues, false)
    end
    #println("Time: ", time, " seconds, Bytes: ", bytes, ", GC time: ", gctime, ", Memory allocations: ", memallocs)
    error  = norm(cond_vec - σ)
    println("With error: ", error)
    print(typeof(δσ))
    step_param = dot((σ_prev - cond_vec), δσ )
    σ_prev = σ
    println("And ideal step length: ",step_param)
end

Step 1 

UndefVarError: UndefVarError: `full_step!` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [36]:
#mode_dict[1].δσ
σ

1089-element Vector{Float64}:
 0.9999999999999998
 1.0
 0.9999999999999996
 1.0000000000000002
 0.9999999999999989
 1.0000000000000007
 1.0000000000000002
 0.9999999999999992
 0.9999999999999996
 1.0000000000000009
 ⋮
 1.0
 1.0000000000000004
 1.0
 1.0000000000000009
 0.9999999999999997
 1.0000000000000007
 0.9999999999999998
 1.0000000000000002
 0.9999999999999997

In [37]:
#σ,δσ,r = full_step!(MC, σ, mode_dict, num_modes, tv, dh, cellvalues, false)

## Plotting (To be done)


In [38]:
# project is to grid and plot with Plots.jl
# I wanna use Plots.jl and not Makie.jl or similar because lateron i want to implement a NN on the grid as a regularizer using Lux.jl

In [39]:
# Project to dictionary (coordinate,value)
#PointEvalHandler(grid, points::AbstractVector{Vec{dim,T}})
# Put values from dictionary into Array.

This is our original and final reconstruction:

In [40]:
# Plot reconstruction and original with Plots.jl here